In [ ]:
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF


apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg


cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip install -q selenium

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(10)


url = "https://www.amazon.co.jp/-/en/KADOKAWA/dp/B0BV5GPZPQ/ref=sr_1_2?keywords=%E3%82%B0%E3%83%A9%E3%83%93%E3%82%A2&qid=1678506342&sr=8-2"
url = "https://www.amazon.co.jp/-/en/%E6%A8%B9%E6%99%BA%E5%AD%90-ebook/dp/B0BXF2Y3F2/ref=sr_1_6?keywords=%E3%82%B0%E3%83%A9%E3%83%93%E3%82%A2&qid=1678506342&sr=8-6"
url = "https://www.amazon.co.jp/-/en/%E4%BC%8A%E7%B9%94-%E3%82%82%E3%81%88/dp/4768317162/ref=sr_1_4?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&keywords=%E3%82%B0%E3%83%A9%E3%83%93%E3%82%A2&qid=1678506990&sr=8-4"
#url = "https://www.amazon.co.jp/-/en/%E9%87%91%E5%AD%90%E6%99%BA%E7%BE%8E/dp/B0BX1XV1YY/ref=sr_1_2?crid=2VYTILWJRLKSU&amp;keywords=%E9%87%91%E5%AD%90%E6%99%BA%E7%BE%8E&amp;qid=1678510695&amp;sprefix=%E9%87%91%E5%AD%90%E6%99%BA%E7%BE%8E%2Caps%2C163&amp;sr=8-2"

try:
    driver.get(url)
    time.sleep(1)
    try:
        title_element = driver.find_element(By.ID, 'productTitle')
    except Exception as e:
        # 年齢認証がある場合
        try:
            buttons = driver.find_elements(By.CSS_SELECTOR, ".a-button-inner")
            links = [a.get_attribute("href") for a in buttons[0].find_elements(By.TAG_NAME, "a")]
            print(links)
            driver.get(links[0])
            time.sleep(1)
        except Exception as e:
            print(e)

    result = dict()
    result["site"] = "Amazon"
    result["url"] = driver.current_url
    result["asin"] = driver.current_url.split("/")[-2]
    title_element = driver.find_element(By.ID, 'productTitle')
    result["title"] = title_element.text

    item_price = 0
    elem = driver.find_element(By.ID, "tmmSwatches")
    # 通常価格検索
    spans = elem.find_elements(By.TAG_NAME, 'span.a-color-base')
    for span in spans:
        try:
            print(span.get_attribute('innerHTML'))
            child_spans = span.find_elements(By.TAG_NAME, 'span.a-size-base, span.a-color-price')
            for child_span in child_spans:
                try:
                    price = re.sub(r"\D", "", child_span.text)
                    if price != None and len(price) >= 3:
                        item_price = price
                        break
                except Exception as e:
                    print(e)
        except Exception as e:
            print(e)

    # 上記で価格が取得できない場合
    if item_price == 0:
        spans = elem.find_elements(By.TAG_NAME, 'span.a-color-secondary')
        for span in spans:
            try:
                child_spans = span.find_elements(By.TAG_NAME, 'span.a-color-secondary')
                for child_span in child_spans:
                    try:
                        price = re.sub(r"\D", "", child_span.text)
                        if price != None and len(price) >= 3:
                            item_price = price
                            break
                    except Exception as e:
                        print(e)
            except Exception as e:
                print(e)

    # 上記２つでも価格が取得できない場合
    if item_price == 0:
        try:
            value = driver.find_element(By.ID, "youPayValue")
            price = re.sub(r"\D", "", value.text)
            if price != None and len(price) >= 3:
                item_price = price
        except Exception as e:
            print(e)

    result["price"] = item_price

    # 結果の確認
    print(result)
except Exception as e:
    print(e)
finally:
    driver.quit()

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="youPayValue"]"}
  (Session info: headless chrome=90.0.4430.212)
Stacktrace:
#0 0x5607eb9f57f9 <unknown>
#1 0x5607eb9953b3 <unknown>
#2 0x5607eb6dd016 <unknown>
#3 0x5607eb71181e <unknown>
#4 0x5607eb7478fb <unknown>
#5 0x5607eb734ded <unknown>
#6 0x5607eb7459e1 <unknown>
#7 0x5607eb734c93 <unknown>
#8 0x5607eb706ce4 <unknown>
#9 0x5607eb7084d2 <unknown>
#10 0x5607eb9c1542 <unknown>
#11 0x5607eb9d0ce7 <unknown>
#12 0x5607eb9d09e4 <unknown>
#13 0x5607eb9d513a <unknown>
#14 0x5607eb9d15b9 <unknown>
#15 0x5607eb9b6e00 <unknown>
#16 0x5607eb9e85d2 <unknown>
#17 0x5607eb9e8778 <unknown>
#18 0x5607eba00a1f <unknown>
#19 0x7f9a60f56609 start_thread
#20 0x7f9a6016a133 clone

{'site': 'Amazon', 'url': 'https://www.amazon.co.jp/-/en/%E4%BC%8A%E7%B9%94-%E3%82%82%E3%81%88/dp/4768317162/ref=sr_1_4?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&keywords=%E3%82%B0%E3%83%A9%E3%83%93%E3%82%A2&qid=1678506990&sr